In [1]:
import json
import nltk

data = open('/home/thejus/Downloads/intents.json').read()
intents = json.loads(data)
intents

ignore_words = ['?','!']

words = []
classes = []
documents = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        
        if intent  ['tag'] not in classes:
            classes.append(intent['tag'])
            
classes
documents
words

FileNotFoundError: [Errno 2] No such file or directory: '/home/thejus/Downloads/intents.json'

In [11]:
from nltk.stem import WordNetLemmatizer
import pickle

lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize (w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
words
print(len(documents),'documents')
print(len(classes),'classes',classes)
len(words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [21]:
import random
import numpy as np

training = []

output_empty = [0]*len(classes)
output_empty

for doc in documents:
    bag = []
    
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

x_train = np.array(list(training[:,0]))
y_train = np.array(list(training[:,1]))

print('Traing data created')

Traing data created


In [22]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(128,input_shape = (len(x_train[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

hist = model.fit(x_train,y_train,epochs=200,batch_size=5,verbose=1)
model.save('chatbox_moodel.h5',hist)

print('model created')

Epoch 1/200
47/47 [==============================] - 1s 20ms/step - loss: 2.2489 - accuracy: 0.0638
Epoch 2/200
47/47 [==============================] - 0s 653us/step - loss: 2.1265 - accuracy: 0.1277
Epoch 3/200
47/47 [==============================] - 0s 646us/step - loss: 2.0775 - accuracy: 0.2340
Epoch 4/200
47/47 [==============================] - 0s 821us/step - loss: 2.0012 - accuracy: 0.3191
Epoch 5/200
47/47 [==============================] - 0s 481us/step - loss: 1.9273 - accuracy: 0.4043
Epoch 6/200
47/47 [==============================] - 0s 541us/step - loss: 1.8034 - accuracy: 0.4043
Epoch 7/200
47/47 [==============================] - 0s 807us/step - loss: 1.6842 - accuracy: 0.4894
Epoch 8/200
47/47 [==============================] - 0s 750us/step - loss: 1.5225 - accuracy: 0.5319
Epoch 9/200
47/47 [==============================] - 0s 491us/step - loss: 1.3768 - accuracy: 0.5745
Epoch 10/200
47/47 [==============================] - 0s 796us/step - loss: 1.2554 - accurac

47/47 [==============================] - 0s 682us/step - loss: 0.0858 - accuracy: 0.9787
Epoch 161/200
47/47 [==============================] - 0s 581us/step - loss: 0.0135 - accuracy: 1.0000
Epoch 162/200
47/47 [==============================] - 0s 695us/step - loss: 0.0270 - accuracy: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 590us/step - loss: 0.0091 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 660us/step - loss: 0.0205 - accuracy: 0.9787
Epoch 165/200
47/47 [==============================] - 0s 593us/step - loss: 0.0422 - accuracy: 0.9787
Epoch 166/200
47/47 [==============================] - 0s 610us/step - loss: 0.0441 - accuracy: 0.9787
Epoch 167/200
47/47 [==============================] - 0s 677us/step - loss: 0.0167 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 579us/step - loss: 0.0071 - accuracy: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 603us/step - loss: 0.0116 - a

In [24]:
from keras.models import load_model

model = load_model('chatbox_moodel.h5')

intents = json.loads(open('/home/thejus/Downloads/intents.json').read())
intents
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

In [53]:
def Cleanup_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def Bow (sentence,words,show_details = True):
    
    sentence_words = Cleanup_sentence(sentence)
    bag = [0]*len(words)
    
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i] = 1
                if show_details:
                    print('Found in bag: %s',w)
    return (np.array(bag))

def Predict_class(sentence,model):
    p = Bow(sentence,words,show_details=False)
    
    res = model.predict(np.array([p]))[0]
    
    ERROR_THRESHOLD = 0.25
    
    results = [[i,r ] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1],reverse=True)
    
    return_list = []
    
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
        
    return return_list

def Get_response(ints,intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag']==tag):
            result = random.choice(i['responses'])
            break
    return list

def Chatbot_response(msg):
    ints = Predict_class(msg,model)
    res = Get_response(ints,intents)
    return res


In [54]:
import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = Chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()



Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/thejus/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-54-3e6f186b72ac>", line 14, in send
    ChatLog.insert(END, "Bot: " + res + '\n\n')
TypeError: can only concatenate str (not "type") to str
